In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


*italicized text*# aZURE

In [ ]:
!pip install azure-cognitiveservices-vision-customvision

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 92kB 5.1MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 


In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import time

In [ ]:
# Replace with valid values
#get the vaules from your azure acc
ENDPOINT = "https://customtraining.cognitiveservices.azure.com/"
training_key = "09369326f2ec40d2828f6d64613bc855"
prediction_key = "82569192148a4e1395b718fbdc4799ed"
prediction_resource_id = "/subscriptions/800e51fe-8ccb-4817-a0d3-3d80d2e88479/resourceGroups/mslearn-faceapi/providers/Microsoft.CognitiveServices/accounts/customTraining"

In [ ]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

# Create Project

In [ ]:
publish_iteration_name = "Fish Detection"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
project = trainer.create_project("My Detection Project", domain_id=obj_detection_domain.id)

Creating project...


In [ ]:
project_id = "b0ce3212-becb-44a2-818a-97fa5a666589"
project = trainer.get_project(project_id, custom_headers=None, raw=False)
project

In [ ]:
project.id

'b0ce3212-becb-44a2-818a-97fa5a666589'

#  Create Tag

In [ ]:
Fish_tag = trainer.create_tag(project.id, "fish")

# Get tags

In [ ]:
# Make two tags in the new project
# Fish_tag = trainer.create_tag(project.id, "fish")
Fish_tag = trainer.get_tags(project.id)[0]

# get files count 
to check if the path is correct

In [ ]:
import os
DIR = '/content/drive/MyDrive/raw_data_2/aug_data' #drive folder path
print (len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])) #number of files 

14058


# convert yolo to azre

In [ ]:
#0_1_7068559267_09d793e356_b
#/content/testinh/
def Get_Regions( mianDir ,filename):
  if os.path.exists( mianDir + filename + ".txt" ):
    TextFile = open( mianDir + filename + ".txt", "r").read()
    out_regions = []
    Regions = TextFile.splitlines()
    for imgRegion in Regions :
      x,y,w,h = imgRegion.split(" ")[1:]
      x,y,w,h = float(x),float(y),float(w),float(h)
      x,y = x-(w/2) , y-(h/2)
      oneRegion =  Region(tag_id=Fish_tag.id, left=x,top=y,width=w,height=h)
      out_regions.append(oneRegion)
    return out_regions

# upload func

In [ ]:
def Upload_Data(tagged_images_with_regions):
  sys.stdout.write("\r" + "uploading ......")
  upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
  if not upload_result.is_batch_successful:
      print("Image batch upload failed.")
      for image in upload_result.images:
          print("Image status: ", image.status)

# Run the uplaod 

In [ ]:
import os
import sys
directory = "/content/drive/MyDrive/raw_data_2/aug_data/" #drive path 
tagged_images_with_regions = []
patch_count = 0
total_count = 0 
for filename in os.listdir(directory):
  #logging the stuates
  total_count += 1
  loggingOut = str(total_count) + "/" + "14058" + "    " + str((total_count/14058 )*100)[:3] + "%" 
  sys.stdout.write("\r" + loggingOut)
  sys.stdout.flush()
    
  patch_count += 1
  if filename.endswith(".jpg") :
    regions = Get_Regions(directory, filename[:-4] )
    with open(directory  + filename , mode="rb") as image_contents:
        tagged_images_with_regions.append(
            ImageFileCreateEntry(name=filename[:-4],
                                 contents=image_contents.read(), regions=regions))
  if patch_count == 64 : # maximum number of images in one upload is 64 
      patch_count = 0
      try:
        Upload_Data(tagged_images_with_regions)
      except:
        print("An exception occurred while uploading")
      
      tagged_images_with_regions = []

uploading ......Image batch upload failed.
Image status:  OK
Image status:  OKDuplicate
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
uploading ......Image batch upload failed.
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OKDuplicate
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image